# Calibration VOI Validation Notebook

This notebook validates the Calibration VOI implementation in voiage by demonstrating its functionality with practical examples.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from voiage.methods.calibration import voi_calibration
from voiage.schema import ParameterSet

## Define a Calibration Study Modeler

First, let's create a function that simulates a calibration study and its impact.

In [ ]:
def simulate_calibration_study(psa_samples, study_design, process_spec):
    """Simulate a calibration study and evaluate economic outcomes.
    
    This is a simplified example. In practice, this would involve:
    - Defining which model parameters are targeted for calibration
    - Specifying the design of the data collection effort
    - Simulating the data that would be obtained
    - Detailing the calibration process
    - Evaluating the decision model with parameters updated via calibration
    """
    # Extract parameters
    if isinstance(psa_samples, ParameterSet):
        params = psa_samples.parameters
    else:
        params = psa_samples
    
    # Get treatment effectiveness
    if 'effectiveness' in params:
        effectiveness = params['effectiveness']
    else:
        effectiveness = np.full_like(next(iter(params.values())), 0.7)
        
    # Get treatment costs
    if 'cost' in params:
        cost = params['cost']
    else:
        cost = np.full_like(next(iter(params.values())), 5000)
    
    # Willingness-to-pay threshold (£ per QALY)
    wtp_threshold = 20000
    
    # Calculate net benefits for each strategy
    n_samples = len(effectiveness)
    n_strategies = 2
    
    net_benefits = np.zeros((n_samples, n_strategies))
    
    # Standard care net benefit
    net_benefits[:, 0] = effectiveness * 0.8 * wtp_threshold - cost * 0.8
    # New treatment net benefit (slightly better)
    net_benefits[:, 1] = effectiveness * 1.0 * wtp_threshold - cost * 1.0
    
    # Create ValueArray
    import xarray as xr
    dataset = xr.Dataset(
        {"net_benefit": (("n_samples", "n_strategies"), net_benefits)},
        coords={
            "n_samples": np.arange(n_samples),
            "n_strategies": np.arange(n_strategies),
            "strategy": ("n_strategies", ["Standard Care", "New Treatment"])
        }
    )
    
    from voiage.schema import ValueArray
    return ValueArray(dataset=dataset)

## Generate Parameter Samples

Let's generate parameter samples for our probabilistic sensitivity analysis (PSA).

In [ ]:
# Generate parameter samples
np.random.seed(42)  # For reproducibility
n_samples = 1000

# Treatment effectiveness (QALYs)
effectiveness = np.random.normal(0.7, 0.1, n_samples)

# Treatment costs (£)
cost = np.random.normal(5000, 500, n_samples)

# Create ParameterSet
parameters = {
    'effectiveness': effectiveness,
    'cost': cost
}

parameter_set = ParameterSet.from_numpy_or_dict(parameters)
print(f"Generated {parameter_set.n_samples} parameter samples")

## Define Calibration Study Design

Let's define the calibration study design.

In [ ]:
# Define calibration study design
calibration_study_design = {
    "experiment_type": "lab",
    "sample_size": 200,
    "variables_measured": ["effectiveness", "cost"],
    "duration": 6,  # months
    "confidence_level": 0.95
}

print("Calibration study design:")
for key, value in calibration_study_design.items():
    print(f"  {key}: {value}")

## Define Calibration Process Specification

Let's define the calibration process specification.

In [ ]:
# Define calibration process specification
calibration_process_spec = {
    "method": "bayesian",
    "likelihood_function": "normal",
    "prior_distribution": "normal",
    "optimization_algorithm": "mcmc",
    "convergence_criteria": 0.01
}

print("Calibration process specification:")
for key, value in calibration_process_spec.items():
    print(f"  {key}: {value}")

## Calculate Calibration VOI

Now let's calculate the Value of Information for our calibration study.

In [ ]:
# Calculate calibration VOI
calibration_voi_value = voi_calibration(
    cal_study_modeler=simulate_calibration_study,
    psa_prior=parameter_set,
    calibration_study_design=calibration_study_design,
    calibration_process_spec=calibration_process_spec,
    n_outer_loops=10
)

print(f"Calibration VOI: £{calibration_voi_value:,.0f}")

## Population-Adjusted Calibration VOI

Let's calculate the population-adjusted Calibration VOI for a larger population.

In [ ]:
# Calculate population-adjusted calibration VOI
calibration_voi_pop = voi_calibration(
    cal_study_modeler=simulate_calibration_study,
    psa_prior=parameter_set,
    calibration_study_design=calibration_study_design,
    calibration_process_spec=calibration_process_spec,
    population=100000,  # 100,000 patients
    time_horizon=10,    # 10 years
    discount_rate=0.03, # 3% discount rate
    n_outer_loops=10
)

print(f"Population-adjusted Calibration VOI: £{calibration_voi_pop:,.0f}")

## Sensitivity Analysis

Let's perform a sensitivity analysis by varying the sample size of the calibration study.

In [ ]:
# Vary sample size and calculate calibration VOI
sample_sizes = [50, 100, 200, 500]
voi_values = []

for sample_size in sample_sizes:
    study_design = {
        "experiment_type": "lab",
        "sample_size": sample_size,
        "variables_measured": ["effectiveness", "cost"],
        "duration": 6,
        "confidence_level": 0.95
    }
    
    voi_val = voi_calibration(
        cal_study_modeler=simulate_calibration_study,
        psa_prior=parameter_set,
        calibration_study_design=study_design,
        calibration_process_spec=calibration_process_spec,
        n_outer_loops=5
    )
    
    voi_values.append(voi_val)
    print(f"Sample size: {sample_size}, Calibration VOI: £{voi_val:,.0f}")

## Visualization

Let's visualize how Calibration VOI changes with sample size.

In [ ]:
# Plot Calibration VOI vs Sample Size
plt.figure(figsize=(10, 6))
plt.plot(sample_sizes, voi_values, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Sample Size')
plt.ylabel('Calibration VOI (£)')
plt.title('Calibration VOI vs Sample Size for Calibration Study')
plt.grid(True, alpha=0.3)
plt.show()

## Comparison with Different Calibration Methods

Let's compare different calibration methods.

In [ ]:
# Compare different calibration methods
methods = ["bayesian", "maximum_likelihood", "bootstrap"]
voi_method_values = []

for method in methods:
    process_spec = {
        "method": method,
        "likelihood_function": "normal",
        "prior_distribution": "normal" if method == "bayesian" else None,
        "optimization_algorithm": "mcmc" if method == "bayesian" else "gradient_descent"
    }
    
    voi_val = voi_calibration(
        cal_study_modeler=simulate_calibration_study,
        psa_prior=parameter_set,
        calibration_study_design=calibration_study_design,
        calibration_process_spec=process_spec,
        n_outer_loops=5
    )
    
    voi_method_values.append(voi_val)
    print(f"Method: {method}, Calibration VOI: £{voi_val:,.0f}")

## Summary

This notebook has demonstrated:

1. How to set up a calibration VOI analysis with voiage
2. How to define calibration study modelers
3. How to calculate VOI for calibration studies
4. How to perform population-adjusted calculations
5. How to conduct sensitivity analysis on sample size
6. How to compare different calibration methods

The voiage calibration VOI implementation provides a flexible framework for evaluating the value of model calibration studies in health economics and other domains.